<a href="https://colab.research.google.com/github/seyed-mohammadreza-mousavi/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/blob/main/TensorflowTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output;
!pip install --quiet gradio
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
!cd TensorFlowTTS
!pip --quiet install /content/TensorFlowTTS/
!pip --quiet install git+https://github.com/repodiac/german_transliterate
!pip --quiet install SoundFile
%cd TensorFlowTTS
!pip install jamo unidecode tensorflow_addons pypinyin g2p_en pyopenjtalk
!pip install pyopenjtalk
import numpy as np
import soundfile as sf
import yaml
import tensorflow as tf
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor
import gradio as gr

clear_output()

# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")
# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")
# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

def inference(text):
      input_ids = processor.text_to_sequence(text)
      # fastspeech inference

      mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
          input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
          speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
          speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
          f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
          energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
      )

      # melgan inference
      audio_before = mb_melgan.inference(mel_before)[0, :, 0]
      audio_after = mb_melgan.inference(mel_after)[0, :, 0]

      # save to file
      sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
      sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")
      return './audio_after.wav'



In [46]:
inputs = gr.inputs.Textbox(lines=5, label="Input Text")
outputs =  gr.outputs.Audio(type="filepath", label="Output Audio")
title = "VERA STARTUP"
description = "Text 2 speech:"
article = "<p style='text-align: center'><a href='https://ruslanmv.com/'> Check out for more at VERA website </a> | <a href='#'>VERA Github</a></p>"

examples = [
 ["Hello. My name is Test and I am testing this for text 2 speech program."],
 ["With Tensorflow 2, we can speed-up training/inference progress, optimizer further by using fake-quantize aware and pruning, make TTS models can be run faster than real-time and be able to deploy on mobile devices or embedded systems."]]

clear_output()

gr.Interface(inference, inputs, outputs, title=title, description=description, article=article, examples=examples).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://806f6924d8d9fd873e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
